# Testing a Pre-Trained Network

Now that we know how to train an LSTM network, let's see try seeing the results on our own input text!

First, we declare some of our hyperparamters. 

Then, we'll load in our data structures.

In [ ]:
numDimensions = 300
maxSeqLength = 250
batchSize = 50
lstmUnits = 64
numClasses = 2
iterations = 100000

In [ ]:
import numpy as np
wordsList = np.load('wordsList.npy').tolist()
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8

wordVectors = np.load('wordVectors.npy')

Next, we'll create our graph. This has to be the same code used to create the model that was trained

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

with tf.name_scope('input') as scope:
    labels = tf.placeholder(tf.float32, [batchSize, numClasses], name='input_labels')
    input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength], name='input_data')

with tf.name_scope('word2vec'):
    data = tf.nn.embedding_lookup(wordVectors,input_data, name='word2vec')

with tf.name_scope('sentiment_network'):
    c1= tf.nn.rnn_cell.LSTMCell(lstmUnits, state_is_tuple=True)
    c1 = tf.nn.rnn_cell.DropoutWrapper(cell=c1)
    c2 = tf.nn.rnn_cell.LSTMCell(lstmUnits, state_is_tuple=True)
    c2 = tf.nn.rnn_cell.DropoutWrapper(cell=c2)
    lstmCell = tf.nn.rnn_cell.MultiRNNCell([c1, c2], state_is_tuple=True)
    value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

    weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]), name='Weights')
    bias = tf.Variable(tf.constant(0.1, shape=[numClasses]), name='Biases')
    value = tf.transpose(value, [1, 0, 2])
    last = tf.gather(value, int(value.get_shape()[0]) - 1)
    prediction = (tf.matmul(last, weight) + bias)

    correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels), name='loss')
    optimizer = tf.train.AdamOptimizer().minimize(loss, name='AdamOptimizer')


Now, we load in the network. 

In [ ]:
sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models/20180203-222319-50-64'))


Before we input our own text, let's first define a couple of functions. The first is a function to make sure the sentence is in the proper format, and the second is a function that obtains the word vectors for each of the words in a given sentence. 

In [ ]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix

And let's create our simple predict function

In [ ]:
def predict(sentence):
    predictedSentiment = sess.run(prediction, {input_data: getSentenceMatrix(sentence)})[0]
    high_positive = high_negative = False
    if (predictedSentiment[0] > 2):
        high_positive = True
    if (predictedSentiment[1] > 2):
        high_negative = True
    if high_positive and high_negative:
        return 0
    elif high_positive:
        return 1
    elif high_negative:
        return -1
    else:
        return 0
    

In [ ]:
predict("Great wonderful time")

In [ ]:
predict("That horrible show was awesome")

# Now for the fun stuff

## Twitter sentiment analyzer

In [ ]:
import tweepy


In [ ]:


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
def print_happy_face():
    print(u'\U0001f604')
def print_neutral_face():
    print(u'\U0001f610')
def print_sad_face():
    print(u'\U00002639')

In [ ]:
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        # Skip if lang is not english
        if status.lang != 'en':
            return
        print(status.text)
        print('-------------------')
        prediction = predict(status.text)
        if prediction == 1:
            print_happy_face()
        elif prediction == 0:
            print_neutral_face()
        else:
            print_sad_face()
        print('-------------------')    
        
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [ ]:
myStream.filter(track=['bitcoin'], async=True)


In [ ]:
myStream.disconnect()

In [ ]:
import pyttsx3
from os import system

# Windows
# engine = pyttsx3.init()
def sayResponseToPrediction(prediction):
    if prediction == 1:
        stmt = "Happy that you're happy"
    elif prediction == 0:
        stmt = "Hmm, I have no feelings towards that either"
    else:
        stmt = "Relax, relax, relax"
    
    # osX
    system('say %s' % stmt)
    # Windiows
    #engine.say(stmt)
    #engine.runAndWait()

In [ ]:
sayResponseToPrediction(predict("I'm feeling sad and angry, but there's always hope"))